# Fish classification with nnets
Using the [fishdb](http://www.fishdb.co.uk/)

### TODO
- data augmentation ?
- other optimizers ?
- comparer avec et sans
  - batch norm
  - batch norm au début
  - dropout
  - regul

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setup phase
We install packages, make all imports, configure modules and download dataset

In [7]:
%%bash
pip install -q pyyaml
pip install tensorflow==2.0.0-beta1
pip install -q tensorflow-gpu==2.0.0-beta1

In [8]:
%%bash
mkdir datas
ls "/content/drive/My Drive/ml/datas"
unzip -q "/content/drive/My Drive/ml/datas/fishdb.zip" -d datas

fishdb.zip
fishes_all.zip
fishes_one_shot.zip
fishes_species.zip
taiwan_db.zip


mkdir: cannot create directory ‘datas’: File exists
replace datas/fishdb/Honeycomb Cowfish/9.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [9]:
%load_ext tensorboard

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
import skimage

from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models
from tensorboard import notebook
from keras import backend as K
from IPython import display

import os, datetime, time, math, pathlib, itertools, random

keras = tf.keras
AUTOTUNE = tf.data.experimental.AUTOTUNE

print(tf.version.VERSION)
print(tf.keras.__version__)
print("GPU Available: ", tf.test.is_gpu_available())

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
2.0.0-beta1
2.2.4-tf
GPU Available:  True


## Constants
This part will define how to build, train, and evaluate the model

In [10]:
#@markdown ## Data
DIR_DATAS = "datas/fishdb" #@param {type:"string"}
LOAD_FROM = ""  #@param ["", "save_dir"] {allow-input: true}
CHECKPOINTS_DIR = "drive/My Drive/ml/weights/fishes_2" #@param {type:"string"}
checkpoint_dir_name = "checkpoint_" + str(int(time.time()))
KEEP_TO_TRAIN = 10 #@param {type:"number"}


#@markdown ## Model configuration
MODEL_TYPE = "conv_3" #@param ["same", "linear", "dense", "conv", "conv_2", "conv_3", "depth_model", "depthwise_skip"]
IMG_SIDE = 128 #@param {type:"slider", min:10, max:300, step:1}
# -> [96, 128, 160, 192, 224]
IMG_SHAPE = (IMG_SIDE, IMG_SIDE, 3)
NB_CLASSES = 1566 #@param {type:"number"}

#@markdown ## Training configuration
NB_EPOCHS = 40 #@param {type:"number"}
BATCH_SIZE = 32 #@param {type:"number"}
TRIPLETS_PER_IMAGE = 10 #@param {type:"number"}
LEARNING_RATE = 0.001 #@param {type:"number"}
L2_REGUL = 1e-4 #@param {type:"number"}

#@markdown ## Evaluation configuration


checkpoint_dir_name = "checkpoint_" + MODEL_TYPE
print("Saving in {} for this session".format(checkpoint_dir_name))
if LOAD_FROM:
  print("Loading weights from checkpoint {}".format(LOAD_FROM))

Saving in checkpoint_conv_3 for this session


## General code
Helper functions

In [0]:
def get_checkpoint_path(suffix=""):
  os.makedirs(os.path.join(CHECKPOINTS_DIR, checkpoint_dir_name), exist_ok=True)
  return os.path.join(
    CHECKPOINTS_DIR,
    checkpoint_dir_name,
    "weights" + suffix + ".hdf5"
  )

In [0]:
class Timer():
  def __init__(self, to_int = True):
    self.t = time.time()
    self.to_int = to_int
  
  def get(self, reset=True):
    t2 = time.time()
    d = t2 - self.t
    if self.to_int:
      d = int(d)
    if reset:
      self.t = t2
    return d

In [0]:
def show_image(image):
	plt.imshow(image)
	plt.show()

## Import datas and pre-processing

In [0]:
def img_to_rgb(image):
  if len(image.shape) == 3 and image.shape[2] == 3:
    return image
  if len(image.shape) == 3:
    image = image.reshape(image.shape[:2])
  return skimage.color.grey2rgb(image)

def reshape_image(image):
  h, w = image.shape[0], image.shape[1]
  scale = min(IMG_SHAPE[0]/h, IMG_SHAPE[1]/w)
  padH = round((IMG_SHAPE[0] / scale - h) / 2)
  padW = round((IMG_SHAPE[1] / scale - w) / 2)

  padShape = ((padH, padH), (padW, padW), (0,0))
  image = skimage.util.pad(image, padShape, 'constant')

  return skimage.transform.resize(image, IMG_SHAPE, mode='symmetric', preserve_range=True)

def preprocess_image(tf_image):
  tf_image = tf.image.decode_image(tf_image)
  # tf_image = tf.image.resize(tf_image, IMG_SHAPE)
  image = tf_image.numpy().astype(float).reshape(tf_image.shape) / 255.0
  image = img_to_rgb(image)
  image = reshape_image(image)
  return image

def load_and_preprocess_image(img_path):
  return preprocess_image(tf.io.read_file(str(img_path)))

def bin2row(cls_id, row_size=NB_CLASSES):
  l = np.zeros(row_size)
  l[cls_id] = 1
  return l

def labels_bin2row(labels):
  return np.array([bin2row(i) for i in labels])

In [0]:
def list_dir_files(path):
  return [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

def get_datas_paths(dir_path):
  fish_classes = sorted(list(os.listdir(dir_path)))
  fish_dirs = [os.path.join(dir_path, cls) for cls in fish_classes]
  # TODO : subdirs
  img_infos = [
      [(os.path.join(cls_path, img_f_name), label) for img_f_name in list_dir_files(cls_path)]
      for cls_path, label in zip(fish_dirs, list(range(len(fish_dirs))))
  ]
  img_infos = list(itertools.chain(*img_infos))
  paths, labels = [[el[i] for el in img_infos] for i in range(2)]
  return paths, labels, fish_classes # len : nb images | nb images | nb classes

def load_datas(dir_path):
  paths, labels, cls_names = get_datas_paths(dir_path)
  images_datas = [load_and_preprocess_image(img_path) for img_path in paths]
  images_datas, labels = np.array(images_datas), np.array(labels)
  return images_datas, labels, cls_names

def get_subsets_per_cls(datas, labels, subsets=[None]):
  nb_cls = max(labels)+1
  subsets_datas = [[() for _ in subsets] for _ in range(nb_cls)]
  ids_per_cls = [[] for _ in range(nb_cls)]
  for i in range(len(datas)):
    ids_per_cls[labels[i]].append(i)
  
  for i_cls in range(nb_cls):
    for i_sub, max_datas in enumerate(subsets):
      if max_datas == None:
        max_datas = len(ids_per_cls[i_cls])
      subsets_datas[i_cls][i_sub] = ids_per_cls[i_cls][:max_datas]
      ids_per_cls[i_cls] = ids_per_cls[i_cls][max_datas:]
  split_datas = [([], []) for _ in subsets]
  for i_cls in range(nb_cls):
    for i_sub in range(len(subsets)):
      for i_img in subsets_datas[i_cls][i_sub]:
        split_datas[i_sub][0].append(datas[i_img])
        split_datas[i_sub][1].append(labels[i_img])
  split_datas = [(np.array(a1), np.array(a2)) for (a1, a2) in split_datas]
  return split_datas

Now, we read all the datas

In [16]:
try:
  _ = train_images
except:
  # train_images, train_labels, train_cls_names = load_datas(DIR_DATAS)
  # test_images, test_labels, test_cls_names = train_images, train_labels, train_cls_names
  # test_images, test_labels, test_cls_names = load_datas(DIR_DATAS, "test")
  # train_labels_bin, test_labels_bin = labels_bin2row(train_labels), labels_bin2row(test_labels)

  all_images, all_labels, train_cls_names = load_datas(DIR_DATAS)
  test_cls_names = train_cls_names
  (train_images, train_labels), (test_images, test_labels) = get_subsets_per_cls(all_images, all_labels, (KEEP_TO_TRAIN, None))

print(all_images.shape, all_labels.shape)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(8620, 128, 128, 3) (8620,)
(6296, 128, 128, 3) (6296,)
(2324, 128, 128, 3) (2324,)


## Functions to feed datas to the network

First, helper functions to :
- get all images classed by label

In [0]:
def get_ids_per_cls(labels):
  ids_per_cls = []
  for i in range(len(labels)):
    while len(ids_per_cls) <= labels[i]:
      ids_per_cls.append([])
    ids_per_cls[labels[i]].append(i)
  return ids_per_cls

def sort_by_distance(l, anchor, only_ids=True):
  l2 = [(dist_fct(el, anchor), i) for i, el in enumerate(l)]
  l2.sort()
  if only_ids:
    return [i for d, i in l2]
  return [l[i] for d, i in l2]

The following function select random triplets to train the NN

In [0]:
def get_triplets_random(images, labels, trunk_model):
  nb_images = len(images)
  ids_per_cls = get_ids_per_cls(labels)
  triplets = []
  
  for i_anchor in range(nb_images):
    same_cls = [i for i in ids_per_cls[labels[i_anchor]] if i != i_anchor]
    for _ in range(TRIPLETS_PER_IMAGE):
      i_positive, i_negative = random.choice(same_cls), i_anchor
      while labels[i_negative] == labels[i_anchor]:
        i_negative = random.randint(0, nb_images-1)
      triplets.append([i_anchor, i_positive, i_negative])
  
  return triplets

This functions try to select triplets better than random ones. At first, we run the NN on all examples. Then, we try to select triplets with a positive far from the anchor, and a negative close to it.

In [0]:
# %%time
NB_CENTERS_PER_IMAGE = 3
# FACT_RANDOM = 3
# FACT_RANDOM_POSITIVE = 2

def get_triplets_dists(images, labels, trunk_model):
  timer = Timer()
  print(np.array(images, dtype=np.float32).shape)
  coords = trunk_model.predict(np.array(images))
  ids_per_cls = get_ids_per_cls(labels)
  centers = [np.mean([coords[i] for i in ids_per_cls[lab]]) for lab in range(len(ids_per_cls))]

  centers_away_from_cls = [[] for _ in range(len(ids_per_cls))]
  for i_cls in range(len(ids_per_cls)):
    centers_sorted = sort_by_distance(centers, centers[i_cls])
    centers_sorted = [i_center for i_center in centers_sorted if i_center != i_cls][:NB_CENTERS_PER_IMAGE]
    centers_away_from_cls[i_cls] = centers_sorted
  
  triplets = []
  useful, unuseful = 0, 0
  for anchor in range(len(images)):
    same_cls = [i for i in ids_per_cls[labels[anchor]] if i != anchor]
    # positives = [random.choice(same_cls) for _ in range(TRIPLETS_PER_IMAGE)]
    positives_order = sort_by_distance([coords[i] for i in same_cls], anchor)[::-1]
    positives = [same_cls[i] for i in positives_order]
    # random.shuffle(positives)

    centers_taken = centers_away_from_cls[labels[anchor]]
    negatives = list(itertools.chain(*[ids_per_cls[i_cls] for i_cls in centers_taken]))
    negatives_order = sort_by_distance([coords[i] for i in negatives], anchor)
    negatives = [negatives[i] for i in negatives_order]
    # negatives = negatives[:FACT_RANDOM*TRIPLETS_PER_IMAGE]
    # random.shuffle(negatives)

    for i in range(TRIPLETS_PER_IMAGE):
      i_positive, i_negative = i%len(positives), i%len(negatives)
      dist_diff = dist_fct(coords[anchor], coords[positives[i_positive]]) - dist_fct(coords[anchor], coords[negatives[i_negative]])
      if dist_diff + MARGIN < 0:
        unuseful += 1
      else:
        useful += 1
        triplets.append([anchor, positives[i_positive], negatives[i_negative]])
      # print(triplets[-1], [labels[j] for j in triplets[-1]])
  
  print("\ntriplets computed", timer.get(), "s", "(useful, unuseful) =", (useful, unuseful), "({:.2f}%)".format(useful / (useful + unuseful) * 100))

  return triplets

# triplets = get_triplets_dists(train_images, train_labels, trunk_model)

Now, we need a function to generate triplets during the training process. This function will be called by `fit_generator`

In [0]:
def create_triplet_generator(images, labels, trunk_model, triplets_getter, batch_size):
  triplets = []
  cur_triplet = 0
  while True:
    if cur_triplet + batch_size > len(triplets):
      triplets = triplets_getter(images, labels, trunk_model)
      random.shuffle(triplets)
      cur_triplet = 0
    
    yield (
      [ np.array([images[triplets[cur_triplet + i_triplet][i_in]] for i_triplet in range(batch_size)]) for i_in in range(3)],
      [0] * batch_size
    )

    cur_triplet += batch_size

## Building model

### Base models

In [0]:
def create_linear_model():
  model = keras.models.Sequential([
    layers.Input(IMG_SHAPE),
    layers.Flatten(),
    layers.Dense(NB_CLASSES, activation='softmax'),
  ], name="linear_model")
  return model

Model with only fully-connected layers

In [0]:
def create_dense_model():
  model = keras.models.Sequential([
    layers.Input(IMG_SHAPE),
    # layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(2048, activation='tanh'),
    layers.Dense(1024, activation='tanh'),
    # layers.Dense(512, activation='relu'),
    layers.Dense(NB_CLASSES, activation='softmax'),
  ], name="dense_model")
  return model

In [0]:
def get_regul():
  return keras.regularizers.l2(L2_REGUL)

Convolutional neural networks

In [0]:
def create_conv_model():
  model = keras.models.Sequential([
    layers.Input(IMG_SHAPE),
    layers.BatchNormalization(),

    layers.Conv2D(20, (5, 5), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(40, (5, 5), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(80, (4, 4), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(4096, activation='tanh'),
    # layers.Dropout(0.2),
    layers.Dense(2048, activation='tanh'),
    # layers.Dropout(0.2),
    layers.Dense(NB_CLASSES, activation='softmax')
  ], name="conv_model")
  return model

def create_conv_2_model():
  model = keras.models.Sequential([
    layers.Input(IMG_SHAPE),
    layers.BatchNormalization(),

    layers.Conv2D(32, (5, 5), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(64, (5, 5), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(128, (4, 4), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(256, (4, 4), activation='relu'),
    layers.MaxPool2D((2, 2)),

    layers.Flatten(),
    layers.Dense(1024, activation='tanh'),
    layers.Dropout(0.2),
    layers.Dense(512, activation='tanh'),
    layers.Dropout(0.2),
    layers.Dense(NB_CLASSES, activation='softmax')
  ], name="conv_2_model")
  return model

def create_conv_3_model():
  model = keras.models.Sequential([
    layers.Input(IMG_SHAPE),
    layers.BatchNormalization(),
    
    # layers.Conv2D(32, (8, 8), activation='relu', padding="same", kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.Conv2D(32, (8, 8), activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(64, (4, 4), activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(128, (4, 4), activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(256, (4, 4), activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.MaxPool2D((2, 2)),
    layers.BatchNormalization(),

    layers.Conv2D(4096, (4, 4), activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),

    # layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    # layers.Dense(4096, activation='tanh'),
    layers.Dense(NB_CLASSES, activation='softmax')
  ], name="conv_3_model")
  return model

### Building model

In [34]:
model = globals()["create_{}_model".format(MODEL_TYPE)]()
model.summary()

Model: "conv_3_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_15 (Batc (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 121, 121, 32)      6176      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 60, 60, 32)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 60, 60, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 57, 57, 64)        32832     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 28, 28, 64)       

If there's a model to restore, we will try to restore weights

In [35]:
if LOAD_FROM:
  if LOAD_FROM == "save_dir":
    LOAD_FROM = get_checkpoint_path()
  print("Load weights from", LOAD_FROM)
  # model.load_weights(LOAD_FROM)
  # model = tf.keras.models.load_model(LOAD_FROM)
  trunk_model.load_weights(LOAD_FROM)
else:
  print("No weights to load")

No weights to load


## Training phase

In [0]:
model.compile(
    # optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    # optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.0001),
    # optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.0000001),
    loss='sparse_categorical_crossentropy',
    # loss='mse',
    metrics=['accuracy']
)

In [0]:
# !rm -R logs/*
# %tensorboard --logdir logs

In [39]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
checkpoint_path = get_checkpoint_path(suffix="_1.1")
print("Saving weights at", checkpoint_path)

callbacks = [
  keras.callbacks.ModelCheckpoint(checkpoint_path, load_weights_on_restart=False),
  # keras.callbacks.TensorBoard(logdir, histogram_freq=1),
]

hist = model.fit(
  train_images, train_labels,
  epochs = 60,
  batch_size=BATCH_SIZE,
  callbacks=callbacks,
  validation_data=(test_images, test_labels)
)

Saving weights at drive/My Drive/ml/weights/fishes_2/checkpoint_conv_3/weights_1.1.hdf5
Train on 6296 samples, validate on 2324 samples
Epoch 1/60
6296/6296 [==============================] - 18s 3ms/sample - loss: 7.3177 - accuracy: 0.0065 - val_loss: 7.1826 - val_accuracy: 0.0043
Epoch 2/60
6296/6296 [==============================] - 10s 2ms/sample - loss: 6.4957 - accuracy: 0.0518 - val_loss: 6.6383 - val_accuracy: 0.0146
Epoch 3/60
6296/6296 [==============================] - 10s 2ms/sample - loss: 5.8628 - accuracy: 0.1164 - val_loss: 6.1657 - val_accuracy: 0.0366
Epoch 4/60
6296/6296 [==============================] - 10s 2ms/sample - loss: 5.3212 - accuracy: 0.2141 - val_loss: 5.9755 - val_accuracy: 0.0379
Epoch 5/60
6296/6296 [==============================] - 10s 2ms/sample - loss: 4.8052 - accuracy: 0.3183 - val_loss: 5.8619 - val_accuracy: 0.0448
Epoch 6/60
6296/6296 [==============================] - 10s 2ms/sample - loss: 4.3249 - accuracy: 0.4287 - val_loss: 5.8125 - val

## Functions to compute / plot stats about trained models

In [0]:
def eval_accuracy(model, images, labels):
  success = [False for _ in labels]
  predictions = model.predict(images, BATCH_SIZE)
  predictions = [p.argmax() for p in predictions]
  success = [a==b for a, b in zip(labels, predictions)]
  acc = sum(success) / len(images)
  return acc, success, predictions

In [0]:
def plot_history_key(histories, key='loss', color=None):
  plt.figure(figsize=(14,8))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' validation '+key, color=color)
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' train '+key)

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])

def plot_history(history, name="", accuracy_key="accuracy"):
  history = [(name, history)]
  plot_history_key(history, "loss", color="blue")
  plot_history_key(history, accuracy_key, color="green")

In [0]:
def plot_hist(arrs):
  plt.figure(figsize=(12,5))
  plt.hist(arrs,
    bins = 60,
    color = ['blue', '#D72F1A'],
    # edgecolor = 'black',
    label=["Same dists", "Diff dists"],
    density=True
  )
  plt.legend(loc='upper right')

  plt.tight_layout()
  plt.show()

In [0]:
def plot_results_per_class(is_success, labels, cls_names):
  n_cls = len(cls_names)
  cls_success, cls_failed = [0]*n_cls, [0]*n_cls

  plt.figure(figsize=(14.5,6))

  for i, succ in enumerate(is_success):
    if succ == True:
      cls_success[labels[i]] += 1
    elif succ == False:
      cls_failed[labels[i]] += 1

  # cls_sum = [max(a+b, 1) for a, b in zip(cls_success, cls_failed)]
  # cls_success = [v / s for v, s in zip(cls_success, cls_sum)]
  # cls_failed = [v / s for v, s in zip(cls_failed, cls_sum)]
  
  ind = np.arange(n_cls)
  width = 0.8 # the width of the bars: can also be len(x) sequence
  rotation = 45 if n_cls < 60 else 90

  p1 = plt.bar(ind, cls_success, width, color="#4CAF50")
  p2 = plt.bar(ind, cls_failed, width, bottom=cls_success, color="#EF5350")

  plt.ylabel('Number of tests')
  plt.xlabel('Fish species')
  plt.title('Number of detection success and failure per fish species')
  plt.xticks(ind, cls_names, rotation=rotation)
  # plt.yticks(np.arange(0, 81, 10))
  plt.legend((p1[0], p2[0]), ('Success', 'Failed'))

  plt.show()

## Display trained model stats

In [0]:
# plot_history(hist)

In [0]:
print("===== TRAINING STATS =====")
accuracy, good_results, predictions = eval_accuracy(model, train_images, train_labels)

print("Accuracy : {}%".format(accuracy*100))
plot_results_per_class(good_results, train_labels, train_cls_names)

In [0]:
print("===== TESTING STATS =====")
accuracy, good_results, predictions = eval_accuracy(model, test_images, test_labels)

print("Accuracy : {}%".format(accuracy*100))
plot_results_per_class(good_results, test_labels, test_cls_names)